# Sémantique

_Ismaël Bonneau_

#### Jouons un peu avec la sémantique...

Ce notebook a simplement pour but de jouer avec la sémantique des mots et de prendre en main et comparer plusieurs modèles de connaissance sur la sémantique.

- Le wordnet de NLTK
- Le wordnet de spacy, qui utilise les données du wordnet de NLTK et qui s'installe avec la commande ***pip install spacy-wordnet***. https://spacy.io/
- voir ce qu'on peut imaginer avec des embeddings de mots, qui n'ont rien à voir avec un wordnet mais qui je pense peuvent lui apporter des choses intéressantes.

In [1]:
import os
import ast

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces, split_alphanum, remove_stopwords

CUSTOM_FILTERS = [strip_tags, strip_punctuation, strip_numeric,
                 strip_multiple_whitespaces, split_alphanum, remove_stopwords]

import spacy
from spacy import displacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = spacy.load('en_core_web_sm')

dataset_path = "data"

from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('embeddings_wiki2017/parameters.bin')

In [2]:
with open(dataset_path + os.sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:queries[d][1] for d in queries}

In [97]:
print("query: ", queries["303"])
print("\nquery: ", queries["309"])
print("\nquery: ", queries["402"])

query:  identify positive accomplishments of the hubble telescope since it was launched in 1991

query:  evidence that rap music has a negative effect on young people

query:  what is happening in the field of behavioral genetics, the study of the relative influence of genetic and environmental factors on an individual's behavior or personality?


In [11]:
model.most_similar("Hubble")

[('Telescope', 0.6764059662818909),
 ('NICMOS', 0.6570490598678589),
 ('redshifts', 0.656196653842926),
 ('redshift', 0.6545217037200928),
 ('XMM-Newton', 0.6439074277877808),
 ('astrometric', 0.6431471705436707),
 ('redshifted', 0.6402177214622498),
 ('ROSAT', 0.6322942972183228),
 ('Gamma-ray', 0.6283991932868958),
 ('Spectrograph', 0.627946674823761)]

In [12]:
model.most_similar("rap")

[('hip-hop', 0.8901282548904419),
 ('rappers', 0.834954023361206),
 ('hop', 0.8304222226142883),
 ('rapping', 0.8249049782752991),
 ('gangsta', 0.8081001043319702),
 ('rapper', 0.798496663570404),
 ('raps', 0.7921053171157837),
 ('rapped', 0.776846706867218),
 ('hiphop', 0.7427882552146912),
 ('Hip-hop', 0.7348631620407104)]

## Avec NLTK

In [59]:
from nltk.corpus import wordnet as wn

#### On peut obtenir les Hyponymes: x est un hyponyme de y si x est un type de y.

In [82]:
for word in preprocess_string(queries["305"], CUSTOM_FILTERS):
    if len(wn.synsets(word)) > 0:
        print(word, "hyponyms: ")
        for hyponym in wn.synsets(word)[0].hyponyms():
            print("\t", hyponym)

passenger hyponyms: 
	 Synset('commuter.n.02')
	 Synset('fare.n.03')
	 Synset('hitchhiker.n.01')
	 Synset('stowaway.n.01')
	 Synset('straphanger.n.02')
vehicles hyponyms: 
	 Synset('bumper_car.n.01')
	 Synset('craft.n.02')
	 Synset('military_vehicle.n.01')
	 Synset('rocket.n.01')
	 Synset('skibob.n.01')
	 Synset('sled.n.01')
	 Synset('steamroller.n.02')
	 Synset('wheeled_vehicle.n.01')


#### Pareil pour les Hypernymes: x est un hypernyme de y si y est un type de x (penser à x est la superclasse de y)

In [83]:
for word in preprocess_string(queries["305"], CUSTOM_FILTERS):
    if len(wn.synsets(word)) > 0:
        print(word, "hypernyms: ")
        for hyponym in wn.synsets(word)[0].hypernyms():
            print("\t", hyponym)

passenger hypernyms: 
	 Synset('traveler.n.01')
vehicles hypernyms: 
	 Synset('conveyance.n.03')


#### entailment: x est un entailment de y si y implique x. Par exemple:

In [99]:
wn.synsets('eat')[0].entailments()

[Synset('chew.v.01'), Synset('swallow.v.01')]

### On peut regarder la différence entre les synsets renvoyés par le wordnet et les "mots les plus proches" dans un espace latent d'un word2vec

In [100]:
wn.synsets("gene")

[Synset('gene.n.01')]

In [7]:
model.most_similar("gene")

[('genes', 0.9043672680854797),
 ('mutations', 0.8115547895431519),
 ('allele', 0.7999595403671265),
 ('X-chromosome', 0.7978461384773254),
 ('protein', 0.7975996136665344),
 ('chromosome', 0.7974574565887451),
 ('alleles', 0.792906641960144),
 ('chromosomal', 0.7871671915054321),
 ('mutation', 0.7850880026817322),
 ('phenotype', 0.7844536304473877)]

## Avec Spacy

#### On peut générer un arbre syntaxique avec les dépendances

In [10]:
displacy.render(nlp(queries['305']), style="dep", jupyter=True, options={'distance': 90})

In [44]:
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

#### On peut faire un peu de NER

In [8]:
doc = nlp('identify positive accomplishments of the Hubble telescope since it was launched in 1991')

for ent in doc.ents:
    print(ent.text, ent.label_)

Hubble FAC
1991 DATE
